In [10]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from docx import Document
from docx.shared import Pt  # For font size
from docx.oxml.ns import qn  # For language customization
from docx.oxml import parse_xml  # For custom XML styling
from docx.shared import RGBColor  # For font color
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
import random
import string
from sentence_transformers import SentenceTransformer, util
import os



In [11]:
#auto parameters initialize
doc = Document()
sentence=''
all_urls=[]
news_websites_url=['https://akharinkhabar.ir',#0
                   'https://nournews.ir',#1
                   'https://isna.ir',#2
                   'https://farsnews.ir',#3
                   'https://www.irna.ir',#4
                   'https://www.iribnews.ir',#5
                   'https://www.mehrnews.com',#6
                   'https://www.tasnimnews.com'#7
                   ]
result_map={}
custom_key=0

In [12]:
#parameters must be initialize 
target_page=1 #google page length
# Fill the 'all these words' field
words=''
# Fill the 'this exact word or phrase' field
exact_word =''
# Fill the 'any of these words' field
any_word=''
# Fill the 'none of these words' field
none_word=''
public_url=''#news_websites_url[0-7]#choose webisite
#path save final file
folder_path=''

In [13]:
chrome_options =  webdriver.ChromeOptions()
title_search_news_list=[]
chrome_options.add_argument("--disable-gpu")  
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2,}
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument("--autoplay-policy=no-user-gesture-required")# Abort the request if it's a video file



In [14]:
def start():
    try:
        service = Service("C:/Users/mehdi/Documents/webdrivers/chromedriver.exe")
        driver = webdriver.Chrome(service=service, options=chrome_options)  
        driver.get('https://www.google.com/advanced_search')
    except Exception as e:

        return str(e)

In [15]:
def advance_search():
    # Fill the 'all these words' field
    all_words = driver.find_element(By.ID, 'xX4UFf')
    all_words.send_keys(words)

    # Fill the 'this exact word or phrase' field
    exact_phrase = driver.find_element(By.ID, 'CwYCWc')
    exact_phrase.send_keys(exact_word)

    # Fill the 'any of these words' field
    any_words = driver.find_element(By.ID, 'mSoczb')
    any_words.send_keys(any_word)

    # Fill the 'none of these words' field
    none_words = driver.find_element(By.ID, 't2dX1c')
    none_words.send_keys(none_word)


    # range_end = driver.find_element(By.NAME, 'as_nhi')  # Note: 'as_nhi' doesn't have an ID but has a name attribute
    # range_end.send_keys('35 kg')
    driver.find_element(By.ID, 'NqEkZd').send_keys(public_url.replace("https://", "").replace("http://", ""))  # Fill 'Website or domain'
    advance_button = driver.find_element(By.XPATH, '//*[@id="s1zaZb"]/div[5]/div[8]/div[2]/input[2]')
    advance_button.click()


In [16]:
def compare_texts(text1, text2):

    try:
        
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        embedding1 = model.encode(text1)
        embedding2 = model.encode(text2)
        similarity = util.cos_sim(embedding1, embedding2)
        similarity_percentage = int(similarity.item() *100)
        return similarity_percentage
    except Exception as e:

        return str(e)





In [17]:
def url_collector():
    # Locate the table containing the page links
    table = driver.find_element(By.CLASS_NAME, "AaVjTc")
    # Find all anchor tags (<a>) in the table
    anchors = table.find_elements(By.TAG_NAME, "a")

    # Extract the URLs from the href attribute and store them in a list
    for anchor in anchors:
        url = anchor.get_attribute("href")
        if url not in all_urls:  # Avoid duplicates
            all_urls.append(url)




In [18]:
def navigate_pages( target_page):
  
    current_page = 1
    try:
        while current_page < target_page:
            # Wait for the next page link to be clickable
            next_page = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[aria-label='Page {current_page + 1}']"))
            )
            url_collector()#collect all google urls
            # Print current status
            
            # Scroll the element into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
            
            # Click the next page
            next_page.click()
            
            # Wait for the page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "AaVjTc"))
            )
            
            current_page += 1
            
        
    except TimeoutException:
        print(f"Timeout while trying to navigate to page {current_page + 1}")
    except NoSuchElementException:
        print(f"Could not find navigation element for page {current_page + 1}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")






In [19]:
def compare(key,text):
    check=False
    if not result_map:
        result_map[key]=text
    else:
        for k,v in result_map.items():
            if compare_texts(v,text)>65:
                check=True
                break
        else:
            if check==False:
                result_map[len(result_map)]=text
                
    # Output
    
    print("Map:", result_map)

In [20]:
def akharinKhabarnews():
    print('akharin')
    sentence=driver.find_element(By.CLASS_NAME, "content_content__sfzd5").text
    return sentence
    print(sentence)
    print('akharin done!')
    

In [21]:
def tasnim():
    sentence=str(driver.find_element(By.CLASS_NAME, "story"))
    return sentence
    print('tasni done!')

In [22]:
def iribnews():
    container= str(driver.find_element(By.CLASS_NAME, "div.news_center_body"))
    dive= str(driver.find_element(By.CLASS_NAME, "div.body.body_media_content_show"))
    sentence=f"{container} \n {dive}"
    return sentence
    print('iribnews done!')

In [23]:
def noornews():
    sentence= driver.find_element(By.XPATH, '//*[@id="div_balance_right"]/section/div').text
    return sentence
    print('noor done!')

In [24]:
def irna():
    elements = driver.find_elements(By.CSS_SELECTOR, ".item-text p")
    sentence=" ".join([element.text for element in elements])
    return sentence
    print('irna done!')

In [25]:
def fars():
    news_text_elements=driver.find_element(By.CSS_SELECTOR, ".n-nqnd1g span.auto-dir-block")
    sentence=" ".join([element.text for element in news_text_elements])
    return sentence
    print('fars done!')

In [26]:
def isna():
    summary = driver.find_element(By.CSS_SELECTOR, ".summary").text
    article_body = driver.find_element(By.CSS_SELECTOR, ".item-text").text
    sentence=f"{summary}\n\n{article_body}"
    return sentence
    print(' isna done!')

In [27]:
def mehrnews():
    sentence=driver.find_element(By.CSS_SELECTOR, 'div.item-body div.item-text').text
    return sentence
    print('mehr done!')
    

In [28]:
def save_sentences_to_files(result_map):
    

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Iterate over the result_map and save each sentence to a file
    for idx, sentence in enumerate(result_map.values()):
        file_name = os.path.join(folder_path, f"{idx}.txt")
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(sentence)
        print(f"Saved file: {file_name}")






In [36]:
def collect_and_save():
    for index, url in enumerate(all_urls):
        print(f"Loading website {index + 1}/{len(all_urls)}: {all_urls[index]}")
        driver.get(url)  # Load the URL
        custom_key = index
        
        # Wait until the page has fully loaded (specific condition can be customized)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))  # Wait for the body tag to load
        )
            
        print(f'public:{public_url}')
        try:
            match public_url:
                case 'https://nournews.ir': compare(custom_key, noornews())
                case 'https://isna.ir': compare(custom_key, isna())
                case 'https://farsnews.ir': compare(custom_key, fars())
                case 'https://www.irna.ir': compare(custom_key, irna())
                case 'https://www.iribnews.ir': compare(custom_key, iribnews())
                case 'https://www.mehrnews.com': compare(custom_key, mehrnews())
                case 'https://www.tasnimnews.com': compare(custom_key, tasnim())
                case 'https://akharinkhabar.ir': compare(custom_key, akharinKhabarnews())
        except Exception as e:
            print(f"Error processing element: {e}")
            continue
            
        print(f"Website {url} fully loaded.")
    else:
        # Save sentences to files
        save_sentences_to_files(result_map=result_map)    

    # Close the browser
    print("All websites processed.")

In [35]:
start()#start the chrome

In [ ]:
advance_search()#search in advance google search

In [ ]:
navigate_pages(target_page)#collect urls

In [ ]:
collect_and_save()#compare and save result